In [1]:
import numpy as np
from numpy import random
random.seed(1307)

In [2]:
def softmax2(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x
def softmax1(x):
    return x/sum(x)
def softmax3(x):
    dis = np.ones(shape=x.shape)
    return dis/sum(dis)


In [3]:
def create_data(data_dir_file):
    f = open(data_dir_file, 'r')
    data = {}
    line = f.readline().split()
    data['n']= int(line[0])  
    data['m']= int(line[1])  
    
    data['count1']= data['m']//2 +1 
    data['count2']= data['m']//4 +1
    data['time'] = data['n']
    
    data['softmax'] = softmax1
    if data['n']> 99:
        data['time'] =300
#         data['softmax'] = softmax2
    
    data['xh'] = np.zeros(shape=(data['n']))
    data['yh'] = np.zeros(shape=(data['n']))
    data['vh'] = np.zeros(shape=(data['n']))
    for i in range(data['n']):
        line =f.readline().split()
        data['xh'][i]=int(line[0])
        data['yh'][i]=int(line[1])
        data['vh'][i]=int(line[2])

    data['xc'] = np.zeros(shape=(data['m']))
    data['yc'] = np.zeros(shape=(data['m']))
    data['vc'] = np.zeros(shape=(data['m']))
    for i in range(data['m']):
        line =f.readline().split()
        data['xc'][i]=int(line[0])
        data['yc'][i]=int(line[1])
        data['vc'][i]=int(line[2])

    f.close()
    return data

In [4]:
filename = '500n800m.txt'
f_i = 'DTHL/'+filename
f_o = 'DTHL/LS/res_'+filename
# filename = 'test_30_72.txt'
# f_i = 'Data/'+ filename
# f_o = 'Data/res_ls_'+filename
data = create_data(f_i)
# data
# data['softmax']([3,0.2,1])

In [5]:
def distance(hub, customer):
    return np.sqrt((data['xh'][hub]-data['xc'][customer])**2+(data['yh'][hub]-data['yc'][customer])**2)

data['nearestHub'] = []
for customer in range(data['m']):
    d = np.array([distance(hub, customer) for hub in range(data['n'])]) 
    top5=[]
    for i in range(min(5,len(d))):
        top5.append(d.argsort()[i])
    data['nearestHub'].append(top5)

In [6]:
def initialSolution(e):
    solution = np.zeros(shape=data["m"])
    for customer in range(data["m"]):
        if random.rand() < e:
            solution[customer]= random.choice(data['nearestHub'][customer])
        else:
            solution[customer]= data['nearestHub'][customer][0]
    return solution

In [7]:
def obj_func(solution):
    totalCost = 0
    penalty = 1
    weight = np.array(data["vh"])  #capacity of hub
    w = np.zeros(shape=(data['n']))
    flag = True
    for customer in range(data["m"]):
        hub = int(solution[customer])
        totalCost+= distance(hub,customer)
        w[hub] += data["vc"][customer]
    
    for hub in range(data["n"]):
        if (w[hub] > weight[hub]):
            penalty *= w[hub] / weight[hub]
            flag = False
    return penalty*totalCost, flag

In [8]:
# def obj_func_2(solution):
#     totalCost = 0
#     penalty = 1
#     weight = np.array(data["vh"])  #capacity of hub
#     w = np.zeros(shape=(data['n']))
#     flag = True
#     for customer in range(data["m"]):
#         hub = int(solution[customer])
#         totalCost+= distance(hub,customer)
#         w[hub] += data["vc"][customer]
    
#     for hub in range(data["n"]):
#         if (w[hub] > weight[hub]):
#             penalty *= w[hub] / weight[hub]
#             flag = False
#     return penalty*totalCost, flag

In [9]:
def getHomeChange(solution):
    distribution = np.zeros(shape=(data['m']))
    penalty = 1
    weight = np.array(data["vh"])  #capacity of hub
    w = np.zeros(shape=(data['n']))
    for customer in range(data["m"]):
        hub = int(solution[customer])
        distribution[customer] = distance(hub, customer) * data["vc"][customer] / weight[hub]
        w[hub] += data["vc"][customer]
    
    for customer in range(data["m"]):
        hub = int(solution[customer])
        if w[hub]>weight[hub]:
            distribution[customer] *= w[hub] / weight[hub]
    c = random.choice(np.arange(data['m']), p= data['softmax'](distribution))
    return c

In [10]:
def changeHubForHome(solution, c, best_cur_obj, flag):
    h = solution[c]
    for hub in data['nearestHub'][c]:
        if hub != h:
            tmp = solution[c]
            solution[c]= hub
            new_obj, fl = obj_func(solution)
            if new_obj < best_cur_obj:
                best_cur_obj = new_obj
                flag = fl
            else:
                solution[c] = tmp
    return solution, best_cur_obj, flag

In [11]:
import time

t = time.time()
cur_sol = initialSolution(0.5)
cur_obj, cur_flag = obj_func(cur_sol)

best_sol = cur_sol
best_obj = cur_obj
best_flag = cur_flag

it = 0
ie = 0
count1 =0
count2 = 0
while True:
    cur_obj_tmp = cur_obj
    c = getHomeChange(cur_sol)
    cur_sol, cur_obj, cur_flag = changeHubForHome(cur_sol, c, cur_obj, cur_flag)
    print("Gen {} : {} {}".format(it,cur_obj,cur_flag))
    
    if cur_obj_tmp == cur_obj:
        count1 += 1
    else:
        count1 = 0
        
    
    if count1 == data['count1']:
        if cur_obj < best_obj:
            best_sol = cur_sol
            best_obj = cur_obj
            best_flag = cur_flag
            count2 = 0
        else:
            count2+=1
        if count2 == data['count2']:
            break
        print('best: ',best_obj, best_flag)
        cur_sol = initialSolution(random.rand())
        cur_obj, cur_flag = obj_func(cur_sol)
        count1 = 0
    it += 1
    if time.time()-t > data['time']:
        if cur_obj < best_obj:
            best_sol = cur_sol
            best_obj = cur_obj
            best_flag = cur_flag
        break
print('best:',best_obj, best_flag)
print(time.time()-t)

Gen 0 : 148963.89006220864 False
Gen 1 : 148844.84108110296 False
Gen 2 : 148684.27947771718 False
Gen 3 : 148684.27947771718 False
Gen 4 : 148614.17406309582 False
Gen 5 : 148614.17406309582 False
Gen 6 : 148493.7590026961 False
Gen 7 : 148315.15334107963 False
Gen 8 : 148154.73493374602 False
Gen 9 : 147866.45857589625 False
Gen 10 : 147866.45857589625 False
Gen 11 : 147866.45857589625 False
Gen 12 : 147866.45857589625 False
Gen 13 : 147866.45857589625 False
Gen 14 : 147866.45857589625 False
Gen 15 : 147866.45857589625 False
Gen 16 : 138444.58278484637 False
Gen 17 : 138444.58278484637 False
Gen 18 : 138400.51024980587 False
Gen 19 : 138400.51024980587 False
Gen 20 : 138232.78743233593 False
Gen 21 : 138232.78743233593 False
Gen 22 : 138232.78743233593 False
Gen 23 : 138080.94045698587 False
Gen 24 : 138008.304963871 False
Gen 25 : 138008.304963871 False
Gen 26 : 131097.25875429274 False
Gen 27 : 130507.51296339114 False
Gen 28 : 130507.51296339114 False
Gen 29 : 130442.39642082059 F

Gen 257 : 4488.70904498007 False
Gen 258 : 4486.248795891167 False
Gen 259 : 4482.662899739046 False
Gen 260 : 4482.662899739046 False
Gen 261 : 4482.662899739046 False
Gen 262 : 4476.212503462098 False
Gen 263 : 4476.212503462098 False
Gen 264 : 4476.212503462098 False
Gen 265 : 4476.212503462098 False
Gen 266 : 4470.7720996062335 False
Gen 267 : 4470.354904101199 False
Gen 268 : 4470.354904101199 False
Gen 269 : 4466.040147308552 False
Gen 270 : 4466.040147308552 False
Gen 271 : 4466.040147308552 False
Gen 272 : 4466.040147308552 False
Gen 273 : 4460.660551667369 False
Gen 274 : 4460.660551667369 False
Gen 275 : 4460.660551667369 False
Gen 276 : 4457.186031804134 False
Gen 277 : 4457.186031804134 False
Gen 278 : 4457.186031804134 False
Gen 279 : 3759.292408109517 False
Gen 280 : 3759.292408109517 False
Gen 281 : 3759.292408109517 False
Gen 282 : 3759.292408109517 False
Gen 283 : 3759.292408109517 False
Gen 284 : 3759.292408109517 False
Gen 285 : 3759.0175834808806 False
Gen 286 : 375

Gen 506 : 2163.460958733503 True
Gen 507 : 2163.460958733503 True
Gen 508 : 2163.460958733503 True
Gen 509 : 2163.460958733503 True
Gen 510 : 2163.460958733503 True
Gen 511 : 2163.460958733503 True
Gen 512 : 2163.460958733503 True
Gen 513 : 2160.3378531078856 True
Gen 514 : 2159.3378531078856 True
Gen 515 : 2159.3378531078856 True
Gen 516 : 2159.3378531078856 True
Gen 517 : 2159.3378531078856 True
Gen 518 : 2159.3378531078856 True
Gen 519 : 2159.3378531078856 True
Gen 520 : 2159.3378531078856 True
Gen 521 : 2159.3378531078856 True
Gen 522 : 2159.3378531078856 True
Gen 523 : 2157.3378531078856 True
Gen 524 : 2153.7520666702585 True
Gen 525 : 2153.7520666702585 True
Gen 526 : 2153.7520666702585 True
Gen 527 : 2150.9881346477587 True
Gen 528 : 2150.9881346477587 True
Gen 529 : 2150.9881346477587 True
Gen 530 : 2150.9881346477587 True
Gen 531 : 2150.9881346477587 True
Gen 532 : 2150.211010497041 True
Gen 533 : 2146.1282479667425 True
Gen 534 : 2146.1282479667425 True
Gen 535 : 2146.1282479

Gen 769 : 2052.9072085722873 True
Gen 770 : 2052.9072085722873 True
Gen 771 : 2052.9072085722873 True
Gen 772 : 2052.9072085722873 True
Gen 773 : 2052.9072085722873 True
Gen 774 : 2048.8081890586946 True
Gen 775 : 2048.8081890586946 True
Gen 776 : 2048.8081890586946 True
Gen 777 : 2048.8081890586946 True
Gen 778 : 2048.8081890586946 True
Gen 779 : 2048.8081890586946 True
Gen 780 : 2048.8081890586946 True
Gen 781 : 2048.8081890586946 True
Gen 782 : 2048.8081890586946 True
Gen 783 : 2048.8081890586946 True
Gen 784 : 2048.8081890586946 True
Gen 785 : 2048.8081890586946 True
Gen 786 : 2048.8081890586946 True
Gen 787 : 2048.8081890586946 True
Gen 788 : 2048.8081890586946 True
Gen 789 : 2048.8081890586946 True
Gen 790 : 2048.8081890586946 True
Gen 791 : 2048.8081890586946 True
Gen 792 : 2048.8081890586946 True
Gen 793 : 2048.8081890586946 True
Gen 794 : 2048.8081890586946 True
Gen 795 : 2048.8081890586946 True
Gen 796 : 2048.8081890586946 True
Gen 797 : 2048.8081890586946 True
Gen 798 : 2048

Gen 1025 : 1981.8386570082635 True
Gen 1026 : 1981.8386570082635 True
Gen 1027 : 1981.8386570082635 True
Gen 1028 : 1981.8386570082635 True
Gen 1029 : 1981.8386570082635 True
Gen 1030 : 1981.8386570082635 True
Gen 1031 : 1980.8778290428143 True
Gen 1032 : 1980.8778290428143 True
Gen 1033 : 1980.8778290428143 True
Gen 1034 : 1976.4056930878146 True
Gen 1035 : 1976.4056930878146 True
Gen 1036 : 1976.4056930878146 True
Gen 1037 : 1974.1630524006953 True
Gen 1038 : 1974.1630524006953 True
Gen 1039 : 1974.1630524006953 True
Gen 1040 : 1974.1630524006953 True
Gen 1041 : 1974.1630524006953 True
Gen 1042 : 1974.1630524006953 True
Gen 1043 : 1973.334625275949 True
Gen 1044 : 1973.334625275949 True
Gen 1045 : 1973.334625275949 True
Gen 1046 : 1973.334625275949 True
Gen 1047 : 1973.334625275949 True
Gen 1048 : 1973.334625275949 True
Gen 1049 : 1973.334625275949 True
Gen 1050 : 1973.334625275949 True
Gen 1051 : 1973.334625275949 True
Gen 1052 : 1973.334625275949 True
Gen 1053 : 1973.334625275949 T

...

Gen 23117 : 1986.7323298038646 True
Gen 23118 : 1986.7323298038646 True
Gen 23119 : 1986.7323298038646 True
Gen 23120 : 1986.7323298038646 True
Gen 23121 : 1986.7323298038646 True
Gen 23122 : 1986.7323298038646 True
Gen 23123 : 1986.7323298038646 True
Gen 23124 : 1986.7323298038646 True
Gen 23125 : 1986.7323298038646 True
Gen 23126 : 1986.7323298038646 True
Gen 23127 : 1986.7323298038646 True
Gen 23128 : 1986.7323298038646 True
Gen 23129 : 1986.7323298038646 True
Gen 23130 : 1986.139970656618 True
Gen 23131 : 1986.139970656618 True
Gen 23132 : 1986.139970656618 True
Gen 23133 : 1986.139970656618 True
Gen 23134 : 1986.139970656618 True
Gen 23135 : 1986.139970656618 True
Gen 23136 : 1986.139970656618 True
Gen 23137 : 1986.139970656618 True
Gen 23138 : 1986.139970656618 True
Gen 23139 : 1986.139970656618 True
Gen 23140 : 1986.139970656618 True
Gen 23141 : 1984.736846419185 True
Gen 23142 : 1984.736846419185 True
Gen 23143 : 1984.736846419185 True
Gen 23144 : 1984.736846419185 True
Gen 231

Gen 23370 : 1969.5107504898317 True
Gen 23371 : 1969.5107504898317 True
Gen 23372 : 1969.5107504898317 True
Gen 23373 : 1969.5107504898317 True
Gen 23374 : 1969.5107504898317 True
Gen 23375 : 1969.5107504898317 True
Gen 23376 : 1969.5107504898317 True
Gen 23377 : 1969.5107504898317 True
Gen 23378 : 1969.5107504898317 True
Gen 23379 : 1969.5107504898317 True
Gen 23380 : 1969.5107504898317 True
Gen 23381 : 1969.5107504898317 True
Gen 23382 : 1969.5107504898317 True
Gen 23383 : 1969.5107504898317 True
Gen 23384 : 1969.5107504898317 True
Gen 23385 : 1969.5107504898317 True
Gen 23386 : 1969.5107504898317 True
Gen 23387 : 1969.5107504898317 True
Gen 23388 : 1969.5107504898317 True
Gen 23389 : 1969.5107504898317 True
Gen 23390 : 1969.5107504898317 True
Gen 23391 : 1969.5107504898317 True
Gen 23392 : 1968.0965369274586 True
Gen 23393 : 1968.0965369274586 True
Gen 23394 : 1968.0965369274586 True
Gen 23395 : 1968.0965369274586 True
Gen 23396 : 1968.0965369274586 True
Gen 23397 : 1968.09653692745

Gen 23611 : 1949.0946656269516 True
Gen 23612 : 1949.0946656269516 True
Gen 23613 : 1949.0946656269516 True
Gen 23614 : 1949.0946656269516 True
Gen 23615 : 1949.0946656269516 True
Gen 23616 : 1947.7848073321204 True
Gen 23617 : 1947.7848073321204 True
Gen 23618 : 1947.7848073321204 True
Gen 23619 : 1947.7848073321204 True
Gen 23620 : 1947.7848073321204 True
Gen 23621 : 1947.7848073321204 True
Gen 23622 : 1947.7848073321204 True
Gen 23623 : 1947.7848073321204 True
Gen 23624 : 1947.7848073321204 True
Gen 23625 : 1947.7848073321204 True
Gen 23626 : 1947.7848073321204 True
Gen 23627 : 1947.7848073321204 True
Gen 23628 : 1947.7848073321204 True
Gen 23629 : 1947.7848073321204 True
Gen 23630 : 1947.7848073321204 True
Gen 23631 : 1947.7848073321204 True
Gen 23632 : 1947.7848073321204 True
Gen 23633 : 1947.7848073321204 True
Gen 23634 : 1947.7848073321204 True
Gen 23635 : 1947.7848073321204 True
Gen 23636 : 1946.0208753096201 True
Gen 23637 : 1946.0208753096201 True
Gen 23638 : 1946.02087530962

Gen 23850 : 1935.3080863332436 True
Gen 23851 : 1935.3080863332436 True
Gen 23852 : 1935.3080863332436 True
Gen 23853 : 1935.3080863332436 True
Gen 23854 : 1935.3080863332436 True
Gen 23855 : 1935.3080863332436 True
Gen 23856 : 1935.3080863332436 True
Gen 23857 : 1935.3080863332436 True
Gen 23858 : 1935.3080863332436 True
Gen 23859 : 1935.3080863332436 True
Gen 23860 : 1935.3080863332436 True
Gen 23861 : 1935.3080863332436 True
Gen 23862 : 1935.3080863332436 True
Gen 23863 : 1935.3080863332436 True
Gen 23864 : 1935.3080863332436 True
Gen 23865 : 1935.3080863332436 True
Gen 23866 : 1935.3080863332436 True
Gen 23867 : 1935.3080863332436 True
Gen 23868 : 1935.3080863332436 True
Gen 23869 : 1935.3080863332436 True
Gen 23870 : 1935.3080863332436 True
Gen 23871 : 1935.3080863332436 True
Gen 23872 : 1935.3080863332436 True
Gen 23873 : 1935.3080863332436 True
Gen 23874 : 1935.3080863332436 True
Gen 23875 : 1935.3080863332436 True
Gen 23876 : 1935.3080863332436 True
Gen 23877 : 1935.30808633324

Gen 24093 : 1927.4796592084972 True
Gen 24094 : 1927.4796592084972 True
Gen 24095 : 1927.4796592084972 True
Gen 24096 : 1927.4796592084972 True
Gen 24097 : 1927.4796592084972 True
Gen 24098 : 1927.4796592084972 True
Gen 24099 : 1927.4796592084972 True
Gen 24100 : 1927.4796592084972 True
Gen 24101 : 1927.4796592084972 True
Gen 24102 : 1927.4796592084972 True
Gen 24103 : 1927.4796592084972 True
Gen 24104 : 1927.4796592084972 True
Gen 24105 : 1927.4796592084972 True
Gen 24106 : 1927.4796592084972 True
Gen 24107 : 1927.4796592084972 True
Gen 24108 : 1927.4796592084972 True
Gen 24109 : 1927.4796592084972 True
Gen 24110 : 1927.4796592084972 True
Gen 24111 : 1927.4796592084972 True
Gen 24112 : 1927.4796592084972 True
Gen 24113 : 1927.4796592084972 True
Gen 24114 : 1927.4796592084972 True
Gen 24115 : 1927.4796592084972 True
Gen 24116 : 1927.4796592084972 True
Gen 24117 : 1927.4796592084972 True
Gen 24118 : 1927.4796592084972 True
Gen 24119 : 1927.4796592084972 True
Gen 24120 : 1927.47965920849

In [12]:
f = open(f_o, "w")
# result = cost(sol)
f.write(str(best_obj) +"\n")
for i in range(data["m"]):
    f.write(str(best_sol[i])+" ")
f.close()
